---
title: "Mag7Alpha"
format:
  html:
    engine: jupyter
execute:
  echo: true
  eval: true
jupyter: python3
---




### **mag7Alpha** {.test}

Walk us through a single trade. Entry signal / exit signal, plus other rules like stop-loss.
Our strategy is market-making on the Mag7 stocks, which includes Alphabet, Amazon, Apple, Meta, Microsoft, NVDIA, and Tesla. For this example, we will choose Microsoft. We will use the Stoikov market-making strategy, so we will need two main equations:
The first calculates the reservation price based on the equation:

![Reservation Price](Equation.png)

s = current market mid price
q = quantity of assets in inventory of base asset (could be positive/negative for long/short positions)
σ = market volatility
T = closing time, when the measurement period ends (conveniently normalized to 1)
t = current time (T is normalized = 1, so t is a time fraction)

The second sets optimal bid-ask spread using the equation:

![Optimal Bid-Ask Spread](BidaskSpread.png)

σ = market volatility
T = closing time, when the measurement period ends (conveniently normalized to 1)
t = current time (T is normalized = 1, so t is a time fraction)
δa, δb = bid/ask spread, symmetrical → δa=δb
γ = inventory risk aversion parameter
κ = order book liquidity parameter

Entry conditions:
We want to create symmetrical bid and ask orders around the market mid-price, but this could lead to the inventory skewing in one direction if there are significant market movements in one direction. The reference price is where the buy and sell orders will be created around.

Can run a few different γ (risk aversion parameter) to determine optimal for P&L.

After calculating reservation price and optimal bid ask spreads:
Bid offer price = reservation price — optimal spread / 2
Ask offer price = reservation price + optimal spread / 2

We then enter into limit orders on both sides of this quote.


Exit conditions:
As the trading day goes on, each parameter of the models will change, and new values for reservation price and optimal spreads will be calculated. We then  set new orders based on the new parameters. This cycle continues indefinitely until the end of our backtesting period.

Stop Loss:
We will use average true range (ATR) to set a stop loss, around 1.5x or 2x ATR. This adjusts our stop loss based on volatility.

Higher ATR → Widen stops to avoid noise and premature execution.
Lower ATR → Tighten stops to lock in profits and exit in cases of volatility spikes.

Data:
We can use ShinyBroker to obtain most of the assets’ attributes, such as price throughout the day, volatility, and timing, via fetch_historical_data. We also need to get technical indicators like ATR, which we can use an external Python library for, like Technical Analysis’ ta.volatility.AverageTrueRange().

Here are the 7 stocks that we trade and their tickers.


#
| Company Name | Ticker Symbol |
|--------------|:--------------|
| Apple              | AAPL   |
| Microsoft          | MSFT   |
| Amazon             | AMZN   |
| Alphabet (Google)  | GOOGL  |
| Meta Platforms     | META   |
| Nvidia             | NVDA   |
| Tesla              | TSLA   |

: Mag7 stocks: Company Name, Ticker Symbol


### Blotter:


In [ ]:
#| echo: false
import pandas as pd
from itables import show
import itables.options as opt

opt.mode = "inline"

# Read the blotter CSV
blotter = pd.read_csv("blotter.csv")

show(blotter)

### Ledger: